Модификация эмбеддингов

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

import numpy as np
import pandas as pd
from typing import Tuple, Dict, List
import random
import re

from torch.utils.data import random_split

torch.__version__

'2.6.0+cu124'

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [7]:
# for precomputed embeddings
from torch import nn
class InputModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=1280, out_features=640)
        self.layer_2 = nn.Linear(in_features=640, out_features=320)
        self.layer_3 = nn.Linear(in_features=320, out_features=64)
        self.relu = nn.ReLU()
    def forward(self, x):
      # Intersperse the ReLU activation function between layers
       return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))

model_0 = InputModel().to(device)
print(model_0)

InputModel(
  (layer_1): Linear(in_features=1280, out_features=640, bias=True)
  (layer_2): Linear(in_features=640, out_features=320, bias=True)
  (layer_3): Linear(in_features=320, out_features=64, bias=True)
  (relu): ReLU()
)


## Tokenizer

In [8]:
tokenizer = KMerTokenizer(5)
nucleotides = "ATCG"
k = 5
vocab = [''.join(kmer) for kmer in itertools.product(nucleotides, repeat=k)]
special_tokens = {'[PAD]': 0, '[SOS]': 1, '[EOS]': 2}
vocab = list(special_tokens.keys()) + vocab
vocab
token_to_id_map = {token: idx for idx, token in enumerate(vocab)}
token_to_id_map
id_to_token_map = {idx: token for idx, token in enumerate(vocab)}
id_to_token_map

NameError: name 'KMerTokenizer' is not defined

In [ ]:
def token_to_id(token: str) -> int:
    """Возвращает ID для одного токена"""
    if token not in token_to_id_map:
        raise ValueError(f"Token '{token}' not in vocabulary")
    return token_to_id_map[token]

token_to_id('[PAD]')

0

In [ ]:
def id_to_token(token_id: int) -> str:
    """Возвращает токен по ID"""
    return id_to_token_map[token_id]
id_to_token(56)

'AAGTT'

In [ ]:
def pad_sequence(sequence: str) -> str:
    """Padding is calculated to make the length a multiple of k"""
    pad_length = (k - len(sequence) % k) % k
    return sequence + '[PAD]' * pad_length


def tokenize(sequence: str) -> List[str]:
    sequence = pad_sequence(sequence)
    step = k
    kmers = []
    for i in range(0, len(sequence) - k + 1, step):
        kmer = sequence[i:i+k]
        if all(c in nucleotides or c == '[PAD]' for c in kmer):
            kmers.append(kmer)
    return kmers

tokenize('AGCTCGCGTGTG')

['AGCTC', 'GCGTG']

In [ ]:
tens = torch.Tensor([4,9,1,3,3])
type(tens)

token_ids = tens.detach().cpu().numpy()

tokens = []
for token_id in token_ids:
    token_id = int(token_id)
    if token_id == eos_id:  # Stop at EOS token
        break
    if token_id not in [pad_id, sos_id]:  # Skip PAD and SOS
        tokens.append(id_to_token_map[token_id])



In [ ]:
pad_id = token_to_id_map['[PAD]']
sos_id = token_to_id_map['[SOS]']
eos_id = token_to_id_map['[EOS]']

def decode(token_ids: torch.Tensor) -> str:
    """Decode a tensor of token IDs back to a sequence string"""
    if isinstance(token_ids, torch.Tensor):
        token_ids = token_ids.detach().cpu().numpy()

    # Convert to integers and remove padding
    tokens = []
    for token_id in token_ids:
        token_id = int(token_id)
        if token_id == eos_id:  # Stop at EOS token
            break
        if token_id not in [pad_id, sos_id]:  # Skip PAD and SOS
            tokens.append(id_to_token_map[token_id])

    # Join k-mers back into sequence
    return ''.join(tokens).replace('[PAD]', '')

decode(tens)

'AAAATAAATCAAAAAAAAAA'

In [ ]:
def encode(sequence: str) -> list[int]:
    """
    Convert a DNA sequence into a list of token IDs.
    Adds [SOS] at the start and [EOS] at the end.
    """
    kmers = tokenize(sequence)
    token_ids = [sos_id] + [token_to_id(kmer) for kmer in kmers] + [eos_id]
    return token_ids

encode('AAAATAAATCAAAAAAAAAA')


[1, 4, 9, 3, 3, 2]

In [9]:
#tokenizer
from typing import List
import itertools

class KMerTokenizer:
    """
    Tokenizer for DNA sequences that splits sequences into k-mers.

    Features:
    - Builds a vocabulary of all possible k-mers for given nucleotides.
    - Supports special tokens: [PAD], [SOS], [EOS].
    - Converts k-mers to integer IDs and vice versa.
    - Can pad sequences so that their length is a multiple of k.
    - Provides methods for tokenizing and decoding sequences.

    Parameters:
    ----------
    k : int
        Length of each k-mer (default: 5).
    nucleotides : str
        Allowed characters in sequences (default: "ATCG").

    Attributes:
    ----------
    vocab : List[str]
        List of all k-mers plus special tokens.
    token_to_id_map : dict
        Maps k-mer strings to integer IDs.
    id_to_token_map : dict
        Maps integer IDs back to k-mer strings.
    pad_id, sos_id, eos_id : int
        IDs of special tokens.
    """
    def __init__(self, k: int = 5, nucleotides: str = "ATCG"):
        self.k = k
        self.nucleotides = nucleotides

        # Строим основной словарь k-меров
        self.vocab = self._build_vocab()

        # Добавляем специальные токены в начало
        self.special_tokens = {'[PAD]': 0, '[SOS]': 1, '[EOS]': 2}
        self.vocab = list(self.special_tokens.keys()) + self.vocab

        # Создаем отображения
        self.token_to_id_map = {token: idx for idx, token in enumerate(self.vocab)}
        self.id_to_token_map = {idx: token for idx, token in enumerate(self.vocab)}

        self.pad_id = self.token_to_id_map['[PAD]']
        self.sos_id = self.token_to_id_map['[SOS]']
        self.eos_id = self.token_to_id_map['[EOS]']

    def _build_vocab(self) -> List[str]:
        """Makes a list of all combinations"""
        return [''.join(kmer) for kmer in itertools.product(self.nucleotides, repeat=self.k)]

    def token_to_id(self, token: str) -> int:
        """Возвращает ID для одного токена"""
        if token not in self.token_to_id_map:
            raise ValueError(f"Token '{token}' not in vocabulary")
        return self.token_to_id_map[token]

    def id_to_token(self, token_id: int) -> str:
        """Возвращает токен по ID"""
        return self.id_to_token_map[token_id]

    def tokenize(self, sequence: str) -> List[str]:
        sequence = self.pad_sequence(sequence)
        step = self.k
        kmers = []
        for i in range(0, len(sequence) - self.k + 1, step):
            kmer = sequence[i:i+self.k]
            if all(c in self.nucleotides or c == '[PAD]' for c in kmer):
                kmers.append(kmer)
        return kmers

    def encode(self, sequence: str) -> list[int]:
        """
        Convert a DNA sequence into a list of token IDs.
        Adds [SOS] at the start and [EOS] at the end.
        """
        kmers = self.tokenize(sequence)
        token_ids = [self.sos_id] + [self.token_to_id(kmer) for kmer in kmers] + [self.eos_id]
        return token_ids    

    def decode(self, token_ids: torch.Tensor) -> str:
        """Decode a tensor of token IDs back to a sequence string"""
        if isinstance(token_ids, torch.Tensor):
            token_ids = token_ids.detach().cpu().numpy()

        # Convert to integers and remove padding
        tokens = []
        for token_id in token_ids:
            token_id = int(token_id)
            if token_id == self.eos_id:  # Stop at EOS token
                break
            if token_id not in [self.pad_id, self.sos_id]:  # Skip PAD and SOS
                tokens.append(self.id_to_token_map[token_id])

        # Join k-mers back into sequence
        return ''.join(tokens).replace('[PAD]', '')

    def pad_sequence(self, sequence: str) -> str:
        """Padding is calculated to make the length a multiple of k"""
        pad_length = (self.k - len(sequence) % self.k) % self.k
        return sequence + '[PAD]' * pad_length

    def __len__(self) -> int:
        """Number of kmer combinations"""
        return len(self.vocab)

In [10]:
def causal_mask(size):
        # Creating a square matrix of dimensions 'size x size' filled with ones
        mask = torch.triu(torch.ones(1, size, size), diagonal = 1).type(torch.int)
        return mask == 0

## Decoder

In [11]:
import math
# Creating Input Embeddings
class InputEmbeddings(nn.Module):
    """Makes an embedding for each token"""
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model # Dimension of vectors
        self.vocab_size = vocab_size # Size of the vocabulary
        self.embedding = nn.Embedding(vocab_size, d_model) # PyTorch layer that converts integer indices to dense embeddings
        
    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model) # Normalizing the variance of the embeddings

sequence = 'ACTGTGAACTGTGA'

tokenizer = KMerTokenizer(5)
d_model = 1280
vocab_size= len(tokenizer)
encoded_string = tokenizer.encode('ACTGTGAACTGTGA')
encoded_string = torch.tensor(encoded_string)
print('Encoded_string', encoded_string, 'Encoded_string_size')
input_emb = InputEmbeddings(d_model, vocab_size)
emb_after_layer = input_emb(encoded_string)
print('New embedding size', emb_after_layer.size(), '\n', emb_after_layer)

Encoded_string tensor([  1, 160, 780,   2]) Encoded_string_size
New embedding size torch.Size([4, 1280]) 
 tensor([[  15.2148,   30.5707,    2.4384,  ...,  -81.9324,   -3.8172,
           36.5908],
        [  64.2728,   26.6368,   21.5305,  ...,   25.8757,   30.5263,
           -6.5959],
        [  38.6706,   -1.5827,   41.9668,  ...,   32.6465,  -41.5385,
           23.3219],
        [  41.3128,  -16.8777,   47.8851,  ...,  -53.0897, -109.1080,
          -38.2167]], grad_fn=<MulBackward0>)


In [12]:
# Creating the Positional Encoding
class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model # Dimensionality of the model
        self.seq_len = seq_len # Maximum sequence length
        self.dropout = nn.Dropout(dropout) # Dropout layer to prevent overfitting
        
        # Creating a positional encoding matrix of shape (seq_len, d_model) filled with zeros
        pe = torch.zeros(seq_len, d_model) 
        
        # Creating a tensor representing positions (0 to seq_len - 1)
        position = torch.arange(0, seq_len, dtype = torch.float).unsqueeze(1) # Transforming 'position' into a 2D tensor['seq_len, 1']
        
        # Creating the division term for the positional encoding formula
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        # Apply sine to even indices in pe
        pe[:, 0::2] = torch.sin(position * div_term)
        # Apply cosine to odd indices in pe
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Adding an extra dimension at the beginning of pe matrix for batch handling
        pe = pe.unsqueeze(0)
        
        # Registering 'pe' as buffer. Buffer is a tensor not considered as a model parameter
        self.register_buffer('pe', pe) 
        
    def forward(self,x):
        # Addind positional encoding to the input tensor X
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
        return self.dropout(x) # Dropout for regularization

seq_len = len(encoded_string)  #len of encoded string
dropout = 0.01
PosEncodeLayer = PositionalEncoding(d_model=d_model, seq_len=seq_len, dropout=dropout)    

emb_after_pos = PosEncodeLayer(emb_after_layer)
print('Old embedding size', emb_after_layer.size(), '\n', emb_after_layer)
print('\nEmbedding after positional encoding', emb_after_pos.size(), '\n', emb_after_pos)


Old embedding size torch.Size([4, 1280]) 
 tensor([[  15.2148,   30.5707,    2.4384,  ...,  -81.9324,   -3.8172,
           36.5908],
        [  64.2728,   26.6368,   21.5305,  ...,   25.8757,   30.5263,
           -6.5959],
        [  38.6706,   -1.5827,   41.9668,  ...,   32.6465,  -41.5385,
           23.3219],
        [  41.3128,  -16.8777,   47.8851,  ...,  -53.0897, -109.1080,
          -38.2167]], grad_fn=<MulBackward0>)

Embedding after positional encoding torch.Size([1, 4, 1280]) 
 tensor([[[  15.3685,   31.8895,    2.4630,  ...,  -81.7499,   -3.8558,
            37.9705],
         [  65.7720,   27.4516,   22.5900,  ...,   27.1471,   30.8347,
            -5.6524],
         [  39.9797,   -2.0190,   43.3209,  ...,   33.9864,  -41.9579,
            24.5676],
         [  41.8726,  -18.0482,   48.5541,  ...,  -52.6159, -110.2098,
           -37.5926]]], grad_fn=<MulBackward0>)


In [13]:
# Creating Layer Normalization
class LayerNormalization(nn.Module):
    
    def __init__(self, eps: float = 10**-6) -> None: # We define epsilon as 0.000001 to avoid division by zero
        super().__init__()
        self.eps = eps
        
        # We define alpha as a trainable parameter and initialize it with ones
        self.alpha = nn.Parameter(torch.ones(1)) # One-dimensional tensor that will be used to scale the input data
        
        # We define bias as a trainable parameter and initialize it with zeros
        self.bias = nn.Parameter(torch.zeros(1)) # One-dimensional tenso that will be added to the input data
        
    def forward(self, x):
        mean = x.mean(dim = -1, keepdim = True) # Computing the mean of the input data. Keeping the number of dimensions unchanged
        std = x.std(dim = -1, keepdim = True) # Computing the standard deviation of the input data. Keeping the number of dimensions unchanged
        
        # Returning the normalized input
        return self.alpha * (x-mean) / (std + self.eps) + self.bias

LayerNormLayer =LayerNormalization()
emb_after_norm = LayerNormLayer(emb_after_pos)
print('Old embedding size', emb_after_pos.size(), '\n', emb_after_pos)
print('\nEmbedding after layer normalization', emb_after_norm.size(), '\n', emb_after_norm)

Old embedding size torch.Size([1, 4, 1280]) 
 tensor([[[  15.3685,   31.8895,    2.4630,  ...,  -81.7499,   -3.8558,
            37.9705],
         [  65.7720,   27.4516,   22.5900,  ...,   27.1471,   30.8347,
            -5.6524],
         [  39.9797,   -2.0190,   43.3209,  ...,   33.9864,  -41.9579,
            24.5676],
         [  41.8726,  -18.0482,   48.5541,  ...,  -52.6159, -110.2098,
           -37.5926]]], grad_fn=<MulBackward0>)

Embedding after layer normalization torch.Size([1, 4, 1280]) 
 tensor([[[ 0.4251,  0.8782,  0.0711,  ..., -2.2389, -0.1023,  1.0450],
         [ 1.8396,  0.7690,  0.6332,  ...,  0.7605,  0.8636, -0.1557],
         [ 1.1057, -0.0636,  1.1987,  ...,  0.9388, -1.1754,  0.6766],
         [ 1.1506, -0.5234,  1.3373,  ..., -1.4891, -3.0981, -1.0694]]],
       grad_fn=<AddBackward0>)


In [14]:
# Creating Feed Forward Layers
class FeedForwardBlock(nn.Module):
    
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        # First linear transformation
        self.linear_1 = nn.Linear(d_model, d_ff) # W1 & b1
        self.dropout = nn.Dropout(dropout) # Dropout to prevent overfitting
        # Second linear transformation
        self.linear_2 = nn.Linear(d_ff, d_model) # W2 & b2
        
    def forward(self, x):
        # (Batch, seq_len, d_model) --> (batch, seq_len, d_ff) -->(batch, seq_len, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

d_ff = 128

FFN_Layer = FeedForwardBlock(d_model=d_model, d_ff=d_ff, dropout=dropout)
emb_after_FFN = FFN_Layer(emb_after_norm)
print('Old embedding size', emb_after_norm.size(), '\n', emb_after_norm)
print('\nEmbedding after Feed Forward', emb_after_FFN.size(), '\n', emb_after_FFN)

Old embedding size torch.Size([1, 4, 1280]) 
 tensor([[[ 0.4251,  0.8782,  0.0711,  ..., -2.2389, -0.1023,  1.0450],
         [ 1.8396,  0.7690,  0.6332,  ...,  0.7605,  0.8636, -0.1557],
         [ 1.1057, -0.0636,  1.1987,  ...,  0.9388, -1.1754,  0.6766],
         [ 1.1506, -0.5234,  1.3373,  ..., -1.4891, -3.0981, -1.0694]]],
       grad_fn=<AddBackward0>)

Embedding after Feed Forward torch.Size([1, 4, 1280]) 
 tensor([[[-0.1161, -0.2812,  0.2438,  ...,  0.1072, -0.2964, -0.3823],
         [-0.1223,  0.0998, -0.2051,  ...,  0.0131,  0.2278, -0.0369],
         [ 0.1268, -0.3058,  0.2368,  ...,  0.1080,  0.0190, -0.2535],
         [ 0.0914,  0.1854, -0.0488,  ...,  0.3302,  0.1793, -0.4427]]],
       grad_fn=<ViewBackward0>)


In [15]:
# Creating the Multi-Head Attention block
class MultiHeadAttentionBlock(nn.Module):
    
    def __init__(self, d_model: int, h: int, dropout: float) -> None: # h = number of heads
        super().__init__()
        self.d_model = d_model
        self.h = h
        
        # We ensure that the dimensions of the model is divisible by the number of heads
        assert d_model % h == 0, 'd_model is not divisible by h'
        
        # d_k is the dimension of each attention head's key, query, and value vectors
        self.d_k = d_model // h # d_k formula, like in the original "Attention Is All You Need" paper
        
        # Defining the weight matrices
        self.w_q = nn.Linear(d_model, d_model) # W_q
        self.w_k = nn.Linear(d_model, d_model) # W_k
        self.w_v = nn.Linear(d_model, d_model) # W_v
        self.w_o = nn.Linear(d_model, d_model) # W_o
        
        self.dropout = nn.Dropout(dropout) # Dropout layer to avoid overfitting
        
    
    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):# mask => When we want certain words to NOT interact with others, we "hide" them
        
        d_k = query.shape[-1] # The last dimension of query, key, and value
        
        # We calculate the Attention(Q,K,V) as in the formula in the image above 
        attention_scores = (query @ key.transpose(-2,-1)) / math.sqrt(d_k) # @ = Matrix multiplication sign in PyTorch
        
        # Before applying the softmax, we apply the mask to hide some interactions between words
        if mask is not None: # If a mask IS defined...
            attention_scores.masked_fill_(mask == 0, -1e9) # Replace each value where mask is equal to 0 by -1e9
        attention_scores = attention_scores.softmax(dim = -1) # Applying softmax
        if dropout is not None: # If a dropout IS defined...
            attention_scores = dropout(attention_scores) # We apply dropout to prevent overfitting
            
        return (attention_scores @ value), attention_scores # Multiply the output matrix by the V matrix, as in the formula
        
    def forward(self, q, k, v, mask): 
        
        query = self.w_q(q) # Q' matrix
        key = self.w_k(k) # K' matrix
        value = self.w_v(v) # V' matrix
        
        
        # Splitting results into smaller matrices for the different heads
        # Splitting embeddings (third dimension) into h parts
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1,2) # Transpose => bring the head to the second dimension
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1,2) # Transpose => bring the head to the second dimension
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1,2) # Transpose => bring the head to the second dimension
        
        # Obtaining the output and the attention scores
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        
        # Obtaining the H matrix
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)
        
        return self.w_o(x) # Multiply the H matrix by the weight matrix W_o, resulting in the MH-A matrix


# Illustration, because we will use encoder block 

h = 8
size = len(encoded_string)
mask = causal_mask(size)


MultiHeadAttentionLayer = MultiHeadAttentionBlock(d_model=d_model, h=h, dropout=dropout)
q, k, v = emb_after_FFN, emb_after_FFN, emb_after_FFN
emb_after_attention = MultiHeadAttentionLayer(q, k, v, mask)
print('Old embedding size', emb_after_FFN.size(), '\n', emb_after_FFN)
print('\nEmbedding after Multi Head Attention', emb_after_attention.size(), '\n', emb_after_attention)


Old embedding size torch.Size([1, 4, 1280]) 
 tensor([[[-0.1161, -0.2812,  0.2438,  ...,  0.1072, -0.2964, -0.3823],
         [-0.1223,  0.0998, -0.2051,  ...,  0.0131,  0.2278, -0.0369],
         [ 0.1268, -0.3058,  0.2368,  ...,  0.1080,  0.0190, -0.2535],
         [ 0.0914,  0.1854, -0.0488,  ...,  0.3302,  0.1793, -0.4427]]],
       grad_fn=<ViewBackward0>)

Embedding after Multi Head Attention torch.Size([1, 4, 1280]) 
 tensor([[[-0.0003, -0.0385, -0.0059,  ...,  0.1888,  0.0159, -0.0928],
         [ 0.0361,  0.0236, -0.0014,  ...,  0.1200, -0.0058, -0.0831],
         [ 0.0408,  0.0293,  0.0095,  ...,  0.1043, -0.0080, -0.0498],
         [ 0.0203,  0.0379, -0.0078,  ...,  0.1200, -0.0067,  0.0007]]],
       grad_fn=<ViewBackward0>)


In [28]:
size = len(encoded_string)
causal_mask(size)

tensor([[[ True, False, False, False],
         [ True,  True, False, False],
         [ True,  True,  True, False],
         [ True,  True,  True,  True]]])

In [17]:
# Building Residual Connection
class ResidualConnection(nn.Module):
    def __init__(self, dropout: float) -> None:
        super().__init__()
        self.dropout = nn.Dropout(dropout) # We use a dropout layer to prevent overfitting
        self.norm = LayerNormalization() # We use a normalization layer 
    
    def forward(self, x, sublayer):
        # We normalize the input and add it to the original input 'x'. This creates the residual connection process.
        return x + self.dropout(sublayer(self.norm(x)))

ResConnectLayer = ResidualConnection(dropout=dropout)

#Illustration
emb_after_res_connect = ResConnectLayer(emb_after_FFN, LayerNormLayer)
print('Embedding after Layer Norm', emb_after_norm.size(), '\n', emb_after_norm)
print('Embedding after FFN', emb_after_FFN.size(), '\n', emb_after_FFN)
print('\nEmbedding after ResidualConnection', emb_after_res_connect.size(), '\n', emb_after_res_connect)

Embedding after Layer Norm torch.Size([1, 4, 1280]) 
 tensor([[[ 0.4251,  0.8782,  0.0711,  ..., -2.2389, -0.1023,  1.0450],
         [ 1.8396,  0.7690,  0.6332,  ...,  0.7605,  0.8636, -0.1557],
         [ 1.1057, -0.0636,  1.1987,  ...,  0.9388, -1.1754,  0.6766],
         [ 1.1506, -0.5234,  1.3373,  ..., -1.4891, -3.0981, -1.0694]]],
       grad_fn=<AddBackward0>)
Embedding after FFN torch.Size([1, 4, 1280]) 
 tensor([[[-0.1161, -0.2812,  0.2438,  ...,  0.1072, -0.2964, -0.3823],
         [-0.1223,  0.0998, -0.2051,  ...,  0.0131,  0.2278, -0.0369],
         [ 0.1268, -0.3058,  0.2368,  ...,  0.1080,  0.0190, -0.2535],
         [ 0.0914,  0.1854, -0.0488,  ...,  0.3302,  0.1793, -0.4427]]],
       grad_fn=<ViewBackward0>)

Embedding after ResidualConnection torch.Size([1, 4, 1280]) 
 tensor([[[-0.5597, -1.3642,  1.1947,  ...,  0.5291, -1.4384, -1.8571],
         [-0.6569,  0.5586, -1.1098,  ...,  0.0841,  1.2586, -0.1895],
         [ 0.6893, -1.7279,  1.3034,  ...,  0.5841,  0.0867

In [18]:
# Building Decoder Block
class DecoderBlock(nn.Module):
    
    # The DecoderBlock takes in two MultiHeadAttentionBlock. One is self-attention, while the other is cross-attention.
    # It also takes in the feed-forward block and the dropout rate
    def __init__(self,  self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)]) # List of three Residual Connections with dropout rate
        
    def forward(self, x, encoder_output, src_mask, tgt_mask):
        
        # Self-Attention block with query, key, and value plus the target language mask
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        
        # The Cross-Attention block using two 'encoder_ouput's for key and value plus the source language mask. It also takes in 'x' for Decoder queries
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        
        # Feed-forward block with residual connections
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

h = 8

#Just checking, in fact no cross-attention

emb_after_FFN

DecoderLayer = DecoderBlock(self_attention_block=MultiHeadAttentionLayer, cross_attention_block=MultiHeadAttentionLayer, 
                            feed_forward_block=FFN_Layer, dropout=dropout)
emb_after_decoder_block = DecoderLayer(emb_after_FFN, emb_after_FFN, mask, mask)
print('Old embedding size', emb_after_FFN.size(), '\n', emb_after_FFN)
print('\nEmbedding after Decoder Block', emb_after_decoder_block.size(), '\n', emb_after_decoder_block)

Old embedding size torch.Size([1, 4, 1280]) 
 tensor([[[-0.1161, -0.2812,  0.2438,  ...,  0.1072, -0.2964, -0.3823],
         [-0.1223,  0.0998, -0.2051,  ...,  0.0131,  0.2278, -0.0369],
         [ 0.1268, -0.3058,  0.2368,  ...,  0.1080,  0.0190, -0.2535],
         [ 0.0914,  0.1854, -0.0488,  ...,  0.3302,  0.1793, -0.4427]]],
       grad_fn=<ViewBackward0>)

Embedding after Decoder Block torch.Size([1, 4, 1280]) 
 tensor([[[-0.3098, -0.7360, -0.2170,  ...,  1.3335, -0.0305, -1.3116],
         [-0.1044,  0.2375, -0.2831,  ...,  0.5708,  0.3478, -0.9672],
         [ 0.0446, -0.0398, -0.0893,  ...,  0.9082,  0.2502, -1.1708],
         [ 0.0972,  0.4107, -0.0541,  ...,  0.9918, -0.0164, -1.1848]]],
       grad_fn=<AddBackward0>)


## Different Attentions

In [20]:
input_emb

InputEmbeddings(
  (embedding): Embedding(1027, 1280)
)

In [19]:
class DotProductAttention(nn.Module):
    def forward(self, decoder_embedding, encoder_embedding):
        # decoder_embedding: [batch, tgt_len, d_model]
        # encoder_embedding: [batch, 1, d_model]
        scores = torch.bmm(decoder_embedding, encoder_embedding.transpose(1, 2))  # [batch, tgt_len, 1]
        weights = torch.sigmoid(scores)  # since only one key
        context = weights * encoder_embedding  # broadcast
        return context


encoder_embedding = torch.rand(1, 1280).unsqueeze(dim=0)
print('encoder_embedding', encoder_embedding.size(), encoder_embedding)
print('decoder_embedding', emb_after_FFN.size(), emb_after_FFN)
DotAttention = DotProductAttention()
DotAttention_result = DotAttention(emb_after_FFN, encoder_embedding)
print('Dot Attention', DotAttention_result, 'with size', DotAttention_result.size())

encoder_embedding torch.Size([1, 1, 1280]) tensor([[[0.9606, 0.3608, 0.1937,  ..., 0.1608, 0.3047, 0.5157]]])
decoder_embedding torch.Size([1, 4, 1280]) tensor([[[-0.1161, -0.2812,  0.2438,  ...,  0.1072, -0.2964, -0.3823],
         [-0.1223,  0.0998, -0.2051,  ...,  0.0131,  0.2278, -0.0369],
         [ 0.1268, -0.3058,  0.2368,  ...,  0.1080,  0.0190, -0.2535],
         [ 0.0914,  0.1854, -0.0488,  ...,  0.3302,  0.1793, -0.4427]]],
       grad_fn=<ViewBackward0>)
Dot Attention tensor([[[0.8422, 0.3164, 0.1699,  ..., 0.1410, 0.2671, 0.4521],
         [0.8095, 0.3041, 0.1633,  ..., 0.1355, 0.2568, 0.4345],
         [0.9600, 0.3606, 0.1936,  ..., 0.1607, 0.3045, 0.5153],
         [0.4442, 0.1669, 0.0896,  ..., 0.0744, 0.1409, 0.2384]]],
       grad_fn=<MulBackward0>) with size torch.Size([1, 4, 1280])


In [20]:
#Addding DotProductAttention
#Need to modify DecoderBlock

class DecoderBlock(nn.Module):
    
    # The DecoderBlock takes in two MultiHeadAttentionBlock. One is self-attention, while the other is cross-attention.
    # It also takes in the feed-forward block and the dropout rate
    def __init__(self,  self_attention_block: MultiHeadAttentionBlock, dot_attention_block: DotProductAttention, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.dot_attention_block = dot_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)]) # List of three Residual Connections with dropout rate
        
    def forward(self, x, encoder_output, tgt_mask):
        
        # Self-Attention block with query, key, and value plus the target language mask
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        
        # The Dot Attention block using 'encoder_ouput' and 'x' for Decoder queries
        x = self.residual_connections[1](x, lambda x: self.dot_attention_block(x, encoder_output))
        
        # Feed-forward block with residual connections
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x


h = 8
#encoder output with size [1, 1, 1280], as it is a full-sequence embedding
encoder_embedding = torch.rand(1, 1280).unsqueeze(dim=0)

DecoderLayer = DecoderBlock(self_attention_block=MultiHeadAttentionLayer, dot_attention_block=DotAttention, 
                            feed_forward_block=FFN_Layer, dropout=dropout)
emb_after_decoder_block = DecoderLayer(x=emb_after_FFN, encoder_output=encoder_embedding, tgt_mask=mask)
print('Old embedding size', emb_after_FFN.size(), '\n', emb_after_FFN)
print('\nEmbedding after Decoder Block', emb_after_decoder_block.size(), '\n', emb_after_decoder_block)

Old embedding size torch.Size([1, 4, 1280]) 
 tensor([[[-0.1161, -0.2812,  0.2438,  ...,  0.1072, -0.2964, -0.3823],
         [-0.1223,  0.0998, -0.2051,  ...,  0.0131,  0.2278, -0.0369],
         [ 0.1268, -0.3058,  0.2368,  ...,  0.1080,  0.0190, -0.2535],
         [ 0.0914,  0.1854, -0.0488,  ...,  0.3302,  0.1793, -0.4427]]],
       grad_fn=<ViewBackward0>)

Embedding after Decoder Block torch.Size([1, 4, 1280]) 
 tensor([[[-0.3521, -0.7395, -0.2564,  ...,  1.1427, -0.1064, -1.1313],
         [-0.0638,  0.1813, -0.3274,  ...,  0.4122,  0.4064, -0.7593],
         [ 0.1552, -0.1815, -0.1909,  ...,  0.8659,  0.6041, -0.6226],
         [-0.0481,  0.3852, -0.0998,  ...,  0.8850,  0.1325, -1.1832]]],
       grad_fn=<AddBackward0>)


In [21]:
# Building Decoder
# A Decoder can have several Decoder Blocks
class Decoder(nn.Module):
    
    # The Decoder takes in instances of 'DecoderBlock'
    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        
        # Storing the 'DecoderBlock's
        self.layers = layers
        self.norm = LayerNormalization() # Layer to normalize the output
        
    def forward(self, x, encoder_output, tgt_mask):
        
        # Iterating over each DecoderBlock stored in self.layers
        for layer in self.layers:
            # Applies each DecoderBlock to the input 'x' plus the encoder output and source and target masks
            x = layer(x, encoder_output, tgt_mask)
        return self.norm(x) # Returns normalized output


N = 3
Layers_for_Decoder = nn.ModuleList([DecoderLayer for _ in range(N)])
#DecoderLayers(x=emb_after_FFN, encoder_output=encoder_embedding, tgt_mask=mask)
DecoderLayers = Decoder(Layers_for_Decoder)
emb_after_decoder= DecoderLayers(x=emb_after_FFN, encoder_output=encoder_embedding, tgt_mask=mask)
print('Old embedding size', emb_after_FFN.size(), '\n', emb_after_FFN)
print('\nEmbedding after Decoder', emb_after_decoder.size(), '\n', emb_after_decoder)

Old embedding size torch.Size([1, 4, 1280]) 
 tensor([[[-0.1161, -0.2812,  0.2438,  ...,  0.1072, -0.2964, -0.3823],
         [-0.1223,  0.0998, -0.2051,  ...,  0.0131,  0.2278, -0.0369],
         [ 0.1268, -0.3058,  0.2368,  ...,  0.1080,  0.0190, -0.2535],
         [ 0.0914,  0.1854, -0.0488,  ...,  0.3302,  0.1793, -0.4427]]],
       grad_fn=<ViewBackward0>)

Embedding after Decoder torch.Size([1, 4, 1280]) 
 tensor([[[-1.3973, -0.9347, -2.4734,  ...,  0.6672,  0.3083, -1.7239],
         [-0.8610, -0.0220, -2.3517,  ..., -0.1132,  0.3384, -1.5628],
         [-1.1049, -0.6534, -2.2298,  ...,  0.2602,  0.2253, -1.5268],
         [-1.2030,  0.4994, -2.0753,  ...,  0.4170,  0.3657, -1.6111]]],
       grad_fn=<AddBackward0>)


In [22]:
# Buiding Linear Layer
class ProjectionLayer(nn.Module):
    def __init__(self, d_model: int, vocab_size: int) -> None: # Model dimension and the size of the output vocabulary
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size) # Linear layer for projecting the feature space of 'd_model' to the output space of 'vocab_size'
    def forward(self, x):
        return torch.log_softmax(self.proj(x), dim = -1) # Applying the log Softmax function to the output


Layer_for_Projection = ProjectionLayer(d_model, vocab_size)
emb_after_projection= Layer_for_Projection(x=emb_after_decoder)
print('Old embedding size', emb_after_decoder.size(), '\n', emb_after_decoder)
print('\nEmbedding after Decoder', emb_after_projection.size(), '\n', emb_after_projection)

Old embedding size torch.Size([1, 4, 1280]) 
 tensor([[[-1.3973, -0.9347, -2.4734,  ...,  0.6672,  0.3083, -1.7239],
         [-0.8610, -0.0220, -2.3517,  ..., -0.1132,  0.3384, -1.5628],
         [-1.1049, -0.6534, -2.2298,  ...,  0.2602,  0.2253, -1.5268],
         [-1.2030,  0.4994, -2.0753,  ...,  0.4170,  0.3657, -1.6111]]],
       grad_fn=<AddBackward0>)

Embedding after Decoder torch.Size([1, 4, 1027]) 
 tensor([[[-7.7890, -6.6919, -6.9386,  ..., -6.6554, -7.0361, -6.0396],
         [-7.8387, -6.9295, -6.9495,  ..., -7.2869, -7.0614, -5.6473],
         [-8.0664, -6.8310, -6.9144,  ..., -7.4933, -6.9336, -6.1586],
         [-8.1514, -6.6775, -6.5304,  ..., -7.3090, -6.8958, -5.8849]]],
       grad_fn=<LogSoftmaxBackward0>)


In [23]:
# Creating the Transformer Architecture
class Transformer(nn.Module):
    
    # This takes in the encoder and decoder, as well the embeddings for the source and target language.
    # It also takes in the Positional Encoding for the source and target language, as well as the projection layer
    def __init__(self, decoder: Decoder, encoder_output: torch.Tensor, tgt_embed: InputEmbeddings, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.decoder = decoder
        self.encoder_output = encoder_output
        self.tgt_embed = tgt_embed
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer
        
    # Decoder
    def decode(self, encoder_output, tgt, tgt_mask):
        tgt = self.tgt_embed(tgt) # Applying target embeddings to the input target language (tgt)
        tgt = self.tgt_pos(tgt) # Applying target positional encoding to the target embeddings
        
        # Returning the target embeddings, the output of the encoder, and both source and target masks
        # The target mask ensures that the model won't 'see' future elements of the sequence
        return self.decoder(tgt, encoder_output, tgt_mask)
    
    # Applying Projection Layer with the Softmax function to the Decoder output
    def project(self, x):
        return self.projection_layer(x)
    
TransformerModel = Transformer(DecoderLayers, encoder_embedding, input_emb, PosEncodeLayer, Layer_for_Projection)
Decoded = TransformerModel.decode(encoder_embedding, encoded_string, mask)
Projected = TransformerModel.project(Decoded)
Projected

tensor([[[-7.5509, -7.0231, -8.0610,  ..., -6.0540, -7.4407, -7.1622],
         [-7.1885, -6.5775, -7.0325,  ..., -8.3703, -7.8483, -7.0974],
         [-7.5792, -6.7682, -7.5208,  ..., -5.9622, -6.1658, -6.3382],
         [-7.6987, -6.6008, -7.3422,  ..., -6.5089, -7.9309, -6.9887]]],
       grad_fn=<LogSoftmaxBackward0>)

In [ ]:
# Building & Initializing Transformer

# Definin function and its parameter, including model dimension, number of encoder and decoder stacks, heads, etc.
def build_transformer(tgt_vocab_size: int, tgt_seq_len: int, d_model: int = 512, N: int = 6, h: int = 8, dropout: float = 0.1, d_ff: int = 2048) -> Transformer:
    ###################################################
    # Creating Embedding layers
    #src_embed = InputEmbeddings(d_model) # Source embedding
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size) # Target language (Target Vocabulary to 512-dimensional vectors)
    
    # Creating Positional Encoding layers
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout) # Positional encoding for the target language embeddings
    
    # Creating DecoderBlocks
    decoder_blocks = [] # Initial list of empty DecoderBlocks
    for _ in range(N): # Iterating 'N' times to create 'N' DecoderBlocks (N = 6)
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout) # Self-Attention
        dot_attention_block = DotProductAttention() # Dot Attention
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout) # FeedForward
        
        # Combining layers into a DecoderBlock
        decoder_block = DecoderBlock(decoder_self_attention_block, dot_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block) # Appending DecoderBlock to the list of DecoderBlocks
        
    # Creating the Encoder and Decoder by using the EncoderBlocks and DecoderBlocks lists
    decoder = Decoder(nn.ModuleList(decoder_blocks))
    
    # Creating projection layer
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size) # Map the output of Decoder to the Target Vocabulary Space
    
    # Creating the transformer by combining everything above
    transformer = Transformer(decoder=decoder, encoder_output=None, tgt_embed=tgt_embed, tgt_pos=tgt_pos, projection_layer=projection_layer)
    
    # Initialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
            
    return transformer # Assembled and initialized Transformer. Ready to be trained and validated!

model = build_transformer(vocab_size, seq_len, d_model, N, h, dropout, d_ff)
model()

In [52]:
src_embedding = torch.randn(1, 1, d_model)     # e.g., encoder output
tgt_tokens = tokenizer.encode('ACTGTGAACTGTGA')
tgt_tokens = torch.tensor(encoded_string)


print('src_embedding', src_embedding)
print('tgt_tokens', tgt_tokens, tgt_tokens.dtype)
print('mask', mask)

output = model.decode(src_embedding, tgt_tokens, mask)
print(output.shape, output, '\n')
output = model.project(output)
print(output, output.shape)  # [1, seq_len, vocab_size]
#TransformerModel.decode(encoder_embedding, encoded_string, mask)

src_embedding tensor([[[ 0.1713,  1.2322, -1.5217,  ..., -1.4737,  1.3360, -0.9503]]])
tgt_tokens tensor([  1, 160, 780,   2]) torch.int64
mask tensor([[[ True, False, False, False],
         [ True,  True, False, False],
         [ True,  True,  True, False],
         [ True,  True,  True,  True]]])
torch.Size([1, 4, 1280]) tensor([[[ 0.1520, -2.4387,  0.2730,  ...,  1.6572,  1.2640,  0.2215],
         [-1.5356, -1.6081,  0.9493,  ...,  1.6949, -0.2167,  0.2325],
         [-0.4040, -0.1200, -0.3266,  ..., -1.1764,  1.1579, -0.3621],
         [-0.4292, -3.6789, -0.4308,  ...,  0.3597,  0.3701, -1.0180]]],
       grad_fn=<AddBackward0>) 

tensor([[[ -8.5185,  -7.9347,  -7.7795,  ...,  -7.8528,  -7.7096,  -7.2856],
         [ -7.9628,  -8.6557,  -7.6951,  ...,  -7.1788,  -8.2424,  -7.3020],
         [ -8.3352,  -5.5107,  -7.2247,  ...,  -8.2690, -10.4006,  -6.7113],
         [ -6.8145,  -9.8773,  -8.1796,  ...,  -7.2183,  -8.0586,  -7.7067]]],
       grad_fn=<LogSoftmaxBackward0>) torch.

/tmp/ipykernel_2091597/1909504865.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_tokens = torch.tensor(encoded_string)


In [ ]:
## still testing
# Looping until the 'max_len', maximum length, is reached
while True:
    if decoder_input.size(1) == max_len:
        break
        
    # Building a mask for the decoder input
    decoder_mask = causal_mask(decoder_input.size(1)).to(device)
    print(decoder_mask)
    # Calculating the output of the decoder
    print('source', source)
    print('decoder_input', decoder_input)
    decoder_input = decoder_input.type(torch.int64)
    print('decoder_input', decoder_input)
    print('decoder_mask', decoder_mask)
    out = model.decode(source, decoder_input, decoder_mask)  #source is sequence embedding, "encoder_input"
    
    # Applying the projection layer to get the probabilities for the next token
    prob = model.project(out[:, -1])
    
    # Selecting token with the highest probability
    _, next_word = torch.max(prob, dim=1)
    decoder_input = torch.cat([decoder_input, torch.empty(1,1). type_as(source).fill_(next_word.item()).to(device)], dim=1)
    
    # If the next token is an End of Sentence token, we finish the loop
    if next_word == eos_idx:
        break
        
decoder_input.squeeze(0) # Sequence of tokens generated by the decoder


In [31]:
# Define function to obtain the most probable next token
def greedy_decode(model, source, tokenizer_tgt, max_len, device):
    # Retrieving the indices from the start and end of sequences of the target tokens
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    # Initializing the decoder input with the Start of Sentence token
    decoder_input = torch.empty(1,1).fill_(sos_idx).to(device)

    # Looping until the 'max_len', maximum length, is reached
    while True:
        if decoder_input.size(1) == max_len:
            break
            
        # Building a mask for the decoder input
        decoder_mask = causal_mask(decoder_input.size(1)).to(device)
        # Calculating the output of the decoder
        decoder_input = decoder_input.type(torch.int64)
        print('source.size()', source.size(), 'decoder_input.size()', decoder_input.size(), 'decoder_mask.size()', decoder_mask.size())
        out = model.decode(source, decoder_input, decoder_mask)  #source is sequence embedding, "encoder_input"
        
        # Applying the projection layer to get the probabilities for the next token
        prob = model.project(out[:, -1])
        
        # Selecting token with the highest probability
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat([decoder_input, torch.empty(1,1).type_as(source).fill_(next_word.item()).to(device)], dim=1)
        
        # If the next token is an End of Sentence token, we finish the loop
        if next_word == eos_idx:
            break
    return decoder_input.squeeze(0)


source = src_embedding
tokenizer_tgt = tokenizer
max_len = len(encoded_string)

out_tensor = greedy_decode(model, src_embedding, tokenizer_tgt, max_len, device)

source.size() torch.Size([1, 1, 1280]) decoder_input.size() torch.Size([1, 1]) decoder_mask.size() torch.Size([1, 1, 1])
source.size() torch.Size([1, 1, 1280]) decoder_input.size() torch.Size([1, 2]) decoder_mask.size() torch.Size([1, 2, 2])
source.size() torch.Size([1, 1, 1280]) decoder_input.size() torch.Size([1, 3]) decoder_mask.size() torch.Size([1, 3, 3])


In [79]:
tokenizer.decode(out_tensor)

'CATCTGTGTGTAACC'

## Validation need to add losses

In [1]:
from data_setup import AptamersDataset, collate_embeddings
from model_1 import *
from utils import KMerTokenizer, levenshtein_distance, visualize_mismatch
from decode_modes import greedy_decode


In [4]:
 
def collate_embeddings(batch):
    """
    Collate function for dict-based dataset samples.
    Stacks tensors and groups other fields into lists.
    """
    # Stack tensors
    embeddings = torch.stack([item['embedding'] for item in batch])
    decoder_inputs = torch.stack([item['decoder_input'] for item in batch])
    decoder_masks = torch.stack([item['decoder_mask'] for item in batch])
    labels = torch.stack([item['label'] for item in batch])

    # Non-tensor metadata
    embedding_paths = [item['embedding_path'] for item in batch]
    ab_names = [item['ab_name'] for item in batch]
    apt_names = [item['apt_name'] for item in batch]
    tg_names = [item['tg_name'] for item in batch]
    ab_seqs = [item['ab_seq'] for item in batch]
    apt_seqs = [item['apt_seq'] for item in batch]
    tg_seqs = [item['tg_seq'] for item in batch]

    return {
        'embedding': embeddings,
        'embedding_path': embedding_paths,
        'decoder_input': decoder_inputs,
        'decoder_mask': decoder_masks,
        'label': labels,
        'ab_name': ab_names,
        'apt_name': apt_names,
        'tg_name': tg_names,
        'ab_seq': ab_seqs,
        'apt_seq': apt_seqs,
        'tg_seq': tg_seqs
    }


# Define function to obtain the most probable next token
def greedy_decode(model, source, tokenizer_tgt, max_len, device):
    # Retrieving the indices from the start and end of sequences of the target tokens
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    # Initializing the decoder input with the Start of Sentence token
    decoder_input = torch.empty(1,1).fill_(sos_idx).to(device)

    if source.dim() == 2:
        source = source.unsqueeze(1)  # make shape [batch, seq_len=1, d_model]
        print('source size in greedy decode changed', source.size())
    # Looping until the 'max_len', maximum length, is reached
    while True:
        if decoder_input.size(1) == max_len:
            break
            
        # Building a mask for the decoder input
        decoder_mask = causal_mask(decoder_input.size(1)).to(device)
        # Calculating the output of the decoder
        decoder_input = decoder_input.type(torch.int64)
        out = model.decode(source, decoder_input, decoder_mask)  #source is sequence embedding, "encoder_input"
        
        # Applying the projection layer to get the probabilities for the next token
        prob = model.project(out[:, -1])
        
        # Selecting token with the highest probability
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat([decoder_input, torch.empty(1,1).type_as(source).fill_(next_word.item()).to(device)], dim=1)
        
        # If the next token is an End of Sentence token, we finish the loop
        if next_word == eos_idx:
            break
    return decoder_input.squeeze(0)



def levenshtein_distance(s1: str, s2: str) -> int:
    """Calculate the Levenshtein distance between two strings"""
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]

def visualize_mismatch(target: str, predicted: str) -> str:
    output_lines = ["TARGET:    " + target]
    mismatch_line = "PREDICTED: "
    pointer_line = "           "

    max_len = max(len(target), len(predicted))
    for i in range(max_len):
        t_char = target[i] if i < len(target) else "-"
        p_char = predicted[i] if i < len(predicted) else "+"

        mismatch_line += p_char
        if t_char != p_char:
            pointer_line += "^"
        else:
            pointer_line += " "

    output_lines.append(mismatch_line)
    output_lines.append(pointer_line)
    return "\n".join(output_lines)

In [5]:
#set embeddings and sequences paths 
data_path = Path("../data/")
#embeddings_path = Path("../../esm/embeds")
embeddings_path = Path.cwd().parent.parent / "esm" / "embeds"  #/mnt/tank/scratch/azaikina/esm/embeds


seq_len = 40
tokenizer = KMerTokenizer(5)
df_path = data_path / '3_checked_intersections.csv'
df = pd.read_csv(df_path, index_col = 0)

dataset = AptamersDataset(df=df, embeddings_path = embeddings_path, seq_len=seq_len, tokenizer=tokenizer)


# Set seed for reproducibility
torch.manual_seed(42)

# Define split sizes
train_size = int(0.9 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
print(len(train_dataset), len(test_dataset))

⚠️ 4 rows skipped (no embedding file found).
15020 1669


In [6]:
train_dataloader_custom = DataLoader(dataset=train_dataset,
                                     batch_size=5,
                                     shuffle=True,
                                     collate_fn=collate_embeddings)

test_dataloader_custom = DataLoader(dataset=test_dataset,
                                    batch_size=1,
                                    shuffle=False,
                                    collate_fn=collate_embeddings)

train_dataloader_custom, test_dataloader_custom

(<torch.utils.data.dataloader.DataLoader at 0x7fc8e0f8aa10>,
 <torch.utils.data.dataloader.DataLoader at 0x7fc8e0f89ea0>)

In [8]:
encoder_input=embeddings.squeeze()
print(encoder_input.size(0))

NameError: name 'embeddings' is not defined

In [109]:
embeddings.unsqueeze(dim=0).size(0)

1

In [105]:
src_embedding

tensor([[[ 0.1713,  1.2322, -1.5217,  ..., -1.4737,  1.3360, -0.9503]]])

In [39]:
for batch in test_dataloader_custom:
    print(batch['embedding'])

tensor([[ 0.0247, -0.0433, -0.0520,  ..., -0.0298, -0.0695, -0.0625],
        [ 0.0698, -0.0269,  0.0053,  ..., -0.1131, -0.0601, -0.0299]])
tensor([[ 0.0511, -0.0160, -0.0115,  ..., -0.0082, -0.0421,  0.0080],
        [ 0.0511, -0.0160, -0.0115,  ..., -0.0082, -0.0421,  0.0080]])
tensor([[ 0.0511, -0.0160, -0.0115,  ..., -0.0082, -0.0421,  0.0080],
        [-0.0448, -0.0851,  0.0059,  ..., -0.1730, -0.0154,  0.0015]])
tensor([[-0.0448, -0.0851,  0.0059,  ..., -0.1730, -0.0154,  0.0015],
        [-0.0448, -0.0851,  0.0059,  ..., -0.1730, -0.0154,  0.0015]])
tensor([[-0.0448, -0.0851,  0.0059,  ..., -0.1730, -0.0154,  0.0015],
        [-0.0448, -0.0851,  0.0059,  ..., -0.1730, -0.0154,  0.0015]])
tensor([[-0.0448, -0.0851,  0.0059,  ..., -0.1730, -0.0154,  0.0015],
        [-0.0448, -0.0851,  0.0059,  ..., -0.1730, -0.0154,  0.0015]])
tensor([[-0.0448, -0.0851,  0.0059,  ..., -0.1730, -0.0154,  0.0015],
        [-0.0448, -0.0851,  0.0059,  ..., -0.1730, -0.0154,  0.0015]])
tensor([[-0.0

KeyboardInterrupt: 

In [7]:
# Defining function to evaluate the model on the validation dataset
# num_examples = 2, two examples per run
def run_validation(model, validation_ds, tokenizer_tgt, max_len, device, print_msg, global_state=None, writer=None, num_examples=2):
    model.eval() # Setting model to evaluation mode
    count = 0 # Initializing counter to keep track of how many examples have been processed
    
    console_width = 80 # Fixed witdh for printed messages
    
    # Creating evaluation loop
    with torch.no_grad(): # Ensuring that no gradients are computed during this process
        for batch in validation_ds:
            count += 1
            #embeddings, paths, ab_names, apt_names, tg_names, ab_seqs, apt_seqs, tg_seqs = batch
            src_embedding = batch['embedding'].to(device)

            # Ensuring that the batch_size of the validation set is 1
            assert src_embedding.size(0) ==  1, 'Batch size must be 1 for validation.'
            
            # Applying the 'greedy_decode' function to get the model's output for the source text of the input batch
            model_out = greedy_decode(model, src_embedding, tokenizer_tgt, max_len, device)
            
            # Retrieving source and target texts from the batch
            source_text = [batch['ab_name'], batch['tg_name'], batch['ab_seq'], batch['tg_seq']]
            target_name = [batch['apt_name']]
            target_text = batch['apt_seq']
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy()) # Decoded, human-readable model output
            
            # Printing results
            print('-'*console_width)
            print(f'SOURCE: {source_text}')
            print(f'TARGET: {target_text}')
            print(f'TARGET NAME: {target_name}')
            print(f'PREDICTED: {model_out_text}')
            
            # After two examples, we break the loop
            if count == num_examples:
                break


sequence = 'ACTGTGAACTGTGA'
tokenizer = KMerTokenizer(5)
d_model = 1280
vocab_size= len(tokenizer)
encoded_string = tokenizer.encode('ACTGTGAACTGTGA')
encoded_string = torch.tensor(encoded_string)
src_embedding = torch.randn(1, 1, d_model)     # e.g., encoder output
source = src_embedding
tokenizer_tgt = tokenizer
#max_len = len(encoded_string)
max_len = 100
N = 6
h = 8
dropout = 0.1
d_ff = 512
model = build_transformer(vocab_size, max_len, d_model, N, h, dropout, d_ff)

run_validation(model, test_dataloader_custom, tokenizer_tgt, max_len, device, print_msg=None, global_state=None, writer=None, num_examples=2)

source size in greedy decode changed torch.Size([1, 1, 1280])
--------------------------------------------------------------------------------
SOURCE: [['7n8h_H_G_F'], ['spike glycoprotein'], ['VQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLEWMGWINPISSGTSYAQTFQGRVTMTSDTSITTAYMELSRLRSDDTAVYYCARAAPFYDFWSGYSYFDYWGQGTLVTV|IVMMQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPRLLIYGASSRATGIPDRFSGSGSGTDFTLTISRLEPEDFAVYYCQQYGSSAWTFGQGTKV'], ['MFVFLVLLPLVSIQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSCMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYRYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTT

## Training step

In [25]:
apt_seqs

['TGTCCATTAACGCCC']

In [10]:
batch = next(iter(test_dataloader_custom))
embeddings, paths, ab_names, apt_names, tg_names, ab_seqs, apt_seqs, tg_seqs = batch
tokenizer.encode(str(apt_seqs))[1:]

[648, 49, 2]

source.size() torch.Size([1, 1, 1280]) decoder_input.size() torch.Size([1, 1]) decoder_mask.size() torch.Size([1, 1, 1])
source.size() torch.Size([1, 1, 1280]) decoder_input.size() torch.Size([1, 2]) decoder_mask.size() torch.Size([1, 2, 2])
source.size() torch.Size([1, 1, 1280]) decoder_input.size() torch.Size([1, 3]) decoder_mask.size() torch.Size([1, 3, 3])

In [11]:
loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))

NameError: name 'proj_output' is not defined

In [17]:
from tqdm import tqdm
def train_step(model: torch.nn.Transformer, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer):
    # Put model in train mode
    model.train()
    
    # Setup train loss and train accuracy values
    train_loss = 0
    total_levenshtein = 0
    total_normalized_lev = 0
    total_sequences = 0

    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc="Training", leave=True)

    # Loop through data loader data batches
    for i, batch in progress_bar:
        #embeddings, paths, ab_names, apt_names, tg_names, ab_seqs, apt_seqs, tg_seqs = batch
        encoder_output = batch['embedding'].to(device)
        ##print('encoder_output', encoder_output)
        decoder_input = torch.tensor(batch['decoder_input']).to(device)
        ##print('decoder_input', decoder_input)
        decoder_mask = batch['decoder_mask'].to(device)
        ##print('decoder_mask', decoder_mask)
        target_text = batch['apt_seq']
        
        # Running tensors through the Transformer
        #decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
        ##print('encoder_output.size()', encoder_output.size(), 'decoder_input.size()', decoder_input.size(), 'decoder_mask.size()', decoder_mask.size())
        if encoder_output.dim() == 2:
            encoder_output = encoder_output.unsqueeze(1)  # make shape [batch, seq_len=1, d_model]
        ##print('source size in greedy decode changed', source.size())
        decoder_output = model.decode(encoder_output, decoder_input, decoder_mask)
        ##print('decoder_output', decoder_output)
        proj_output = model.project(decoder_output)
        ##print('proj_output', proj_output.size(), proj_output)
        pred_ids = proj_output.argmax(dim=-1).detach().cpu().numpy()
        model_out_text = [tokenizer_tgt.decode(ids) for ids in pred_ids]
        pred_seq = model_out_text[0]  # get the first (and only) sequence
        target_seq = target_text[0]   # if target_text is a list


        label = torch.tensor(batch['label']).to(device)
        ##print('label', label.size(), label)
        ##print('proj_output_view', proj_output.view(-1, len(tokenizer)))
        ##print('label_view', label.view(-1))
        loss = loss_fn(proj_output.view(-1, len(tokenizer)), label.view(-1))
        print('loss', loss)
        lev_dist = levenshtein_distance(pred_seq, target_seq)
        normalized_lev = lev_dist / max(len(target_seq), 1)

        total_levenshtein += lev_dist
        total_normalized_lev += normalized_lev
        total_sequences += 1
        tqdm.write(f'LEVENSTEIN:{lev_dist}')
        tqdm.write(f'NORM_LEVENSTEIN:{normalized_lev:.2f}')

        optimizer.zero_grad()

        loss.backward()
        
        optimizer.step()

        train_loss += loss.item()
    train_loss = train_loss / len(dataloader)
    avg_levenshtein = total_levenshtein / len(dataloader)
    avg_normalized_lev = total_normalized_lev / len(dataloader)
    return train_loss, avg_levenshtein, avg_normalized_lev

    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, eps = 1e-9)
loss_fn = nn.CrossEntropyLoss(ignore_index = tokenizer.token_to_id('[PAD]'), label_smoothing = 0.1).to(device)

train_step(model, train_dataloader_custom, loss_fn, optimizer)


Training:   0%|          | 0/3004 [00:00<?, ?it/s]

/tmp/ipykernel_2392007/1113197593.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  decoder_input = torch.tensor(batch['decoder_input']).to(device)
/tmp/ipykernel_2392007/1113197593.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(batch['label']).to(device)
Training:   0%|          | 0/3004 [00:00<?, ?it/s]

loss tensor(5.2944, grad_fn=<AddBackward0>)
LEVENSTEIN:84
NORM_LEVENSTEIN:1.00


Training:   0%|          | 1/3004 [00:00<23:17,  2.15it/s]

loss tensor(8.5367, grad_fn=<AddBackward0>)
LEVENSTEIN:168
NORM_LEVENSTEIN:4.54


...

loss tensor(6.1741, grad_fn=<AddBackward0>)
LEVENSTEIN:63
NORM_LEVENSTEIN:0.93


Training:   2%|▏         | 51/3004 [00:16<14:46,  3.33it/s]

loss tensor(3.1131, grad_fn=<AddBackward0>)
LEVENSTEIN:22
NORM_LEVENSTEIN:0.26


Training:   2%|▏         | 52/3004 [00:16<15:41,  3.14it/s]


KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm
def test_step(model: torch.nn.Transformer, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              print_msg = None):
    model.eval()
    #progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc="Testing", leave=True)
    # Setup test loss and test accuracy values
    test_loss = 0
    total_levenshtein = 0  # Track total Levenshtein distance
    total_sequences = 0     # Count of sequences processed
    total_normalized_lev = 0
    all_sequences_list = []
    count = 0

    with torch.inference_mode():
    # Loop through data loader data batches
        for batch in progress_bar:
            #embeddings, paths, ab_names, apt_names, tg_names, ab_seqs, apt_seqs, tg_seqs = batch
            encoder_output = batch['embedding'].to(device)
            ##print('encoder_output', encoder_output)
            decoder_input = torch.tensor(batch['decoder_input']).to(device)
            ##print('decoder_input', decoder_input)
            decoder_mask = batch['decoder_mask'].to(device)
            ##print('decoder_mask', decoder_mask)
            
            # Running tensors through the Transformer
            #decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
            ##print('encoder_output.size()', encoder_output.size(), 'decoder_input.size()', decoder_input.size(), 'decoder_mask.size()', decoder_mask.size())
            if encoder_output.dim() == 2:
                encoder_output = encoder_output.unsqueeze(1)  # make shape [batch, seq_len=1, d_model]
            ##print('source size in greedy decode changed', source.size())
            decoder_output = model.decode(encoder_output, decoder_input, decoder_mask)
            ##################model_out = greedy_decode(model, src_embedding, tokenizer_tgt, max_len, device)
            ##print('decoder_output', decoder_output)
            proj_output = model.project(decoder_output)
            ##print('proj_output', proj_output.size(), proj_output)

            label = torch.tensor(batch['label']).to(device)
            ##print('label', label.size(), label)
            ##print('proj_output_view', proj_output.view(-1, len(tokenizer)))
            ##print('label_view', label.view(-1))

            pred_ids = proj_output.argmax(dim=-1).detach().cpu().numpy()
            model_out_text = [tokenizer_tgt.decode(ids) for ids in pred_ids]
            # Retrieving source and target texts from the batch
            source_text = [batch['ab_name'], batch['tg_name'], batch['ab_seq'], batch['tg_seq']]
            target_name = [batch['apt_name']]
            target_text = batch['apt_seq']
            ##################model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy()) # Decoded, human-readable model output
            
            # Printing results
            #print('-'*console_width)
            tqdm.write(f'SOURCE: {source_text}')
            tqdm.write(f'TARGET: {target_text}')
            tqdm.write(f'TARGET NAME: {target_name}')
            tqdm.write(f'PREDICTED: {model_out_text}')
            
            # # After two examples, we break the loop
            # if count == num_examples:
            #     break

            loss = loss_fn(proj_output.view(-1, len(tokenizer)), label.view(-1))
            tqdm.write(f'loss {loss}')

            

            pred_seq = model_out_text[0]  # get the first (and only) sequence
            target_seq = target_text[0]   # if target_text is a list
            tqdm.write(visualize_mismatch(target_seq, pred_seq))

            lev_dist = levenshtein_distance(pred_seq, target_seq)
            normalized_lev = lev_dist / max(len(target_seq), 1)

            total_levenshtein += lev_dist
            total_normalized_lev += normalized_lev
            total_sequences += 1
            tqdm.write(f'LEVENSTEIN:{lev_dist}')
            tqdm.write(f'NORM_LEVENSTEIN:{normalized_lev:.2f}')
            count += 1
            all_sequences_list.append(model_out_text)
            test_loss += loss.item()
        test_loss = test_loss / len(dataloader)
        avg_levenshtein = total_levenshtein / len(dataloader)
        avg_normalized_lev = total_normalized_lev / len(dataloader)
        return test_loss, avg_levenshtein, avg_normalized_lev
    
progress_bar = tqdm(test_dataloader_custom, total=len(test_dataloader_custom), desc="Testing", leave=True)
loss_fn = nn.CrossEntropyLoss(ignore_index = tokenizer.token_to_id('[PAD]'), label_smoothing = 0.1).to(device)
test_step(model, test_dataloader_custom, loss_fn, print_msg=lambda msg: progress_bar.write(msg))

Testing:   0%|          | 0/1669 [00:00<?, ?it/s]/tmp/ipykernel_2392007/2618401485.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  decoder_input = torch.tensor(batch['decoder_input']).to(device)
/tmp/ipykernel_2392007/2618401485.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(batch['label']).to(device)
Testing:   0%|          | 2/1669 [00:00<01:50, 15.14it/s]

SOURCE: [['7n8h_H_G_F'], ['spike glycoprotein'], ['VQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLEWMGWINPISSGTSYAQTFQGRVTMTSDTSITTAYMELSRLRSDDTAVYYCARAAPFYDFWSGYSYFDYWGQGTLVTV|IVMMQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPRLLIYGASSRATGIPDRFSGSGSGTDFTLTISRLEPEDFAVYYCQQYGSSAWTFGQGTKV'], ['MFVFLVLLPLVSIQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSCMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYRYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPGSASSVASQSIIAYTMSLGAENSVAYSNNSIAIPT

Testing:   0%|          | 8/1669 [00:00<01:40, 16.59it/s]

SOURCE: [['1uac_H_L_Y'], ['lysozyme c'], ['DVQLQESGPSLVKPSQTLSLTCSVTGDSITSDYWSWIRKFPGNRLEYMGYVSSFGSTFYNPSLKSRISITRDTSKNQYYLDLNSVTTEDTATYYCANWDGDYWGQGTLVTVSAA|DIVLTQSPATLSVTPGNSVSLSCRASQSIGNNLHWYQQKSHESPRLLIKYASQSISGIPSRFSGSGSGTDFTLSINSVETEDFGMYFCQQSNSWPYTFGGGTKLEIK'], ['KVYGRCELAAAMKRLGLDNYRGYSLGNWVCAAKFESNFNTHATNRNTDGSTDYGILQINSRWWCNDGRTPGSKNLCNIPCSALLSSDITASVNCAKKIASGGNGMNAWVAWRNRCKGTDVHAWIRGCRL']]
TARGET: ['UGGUCUCCCAAUUCUAAACUUUCUCCAUCGUAUCUGGG']
TARGET NAME: [['APIPred_1751']]
PREDICTED: ['TACTTAGGCAAGGCACAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGAGGCAAGGCAAGGCAAGGCAAGGCAAGGCACAGTGCAGTGCAGTGCAGTG']
loss 6.469664096832275
TARGET:    UGGUCUCCCAAUUCUAAACUUUCUCCAUCGUAUCUGGG
PREDICTED: TACTTAGGCAAGGCACAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGCAGTGAGGCAAGGCAAGGCAAGGCAAGGCAAGGCACAGTGCAGTGCAGTGCAGTG
           ^^^^^^^^   

Testing:   1%|          | 10/1669 [00:00<01:44, 15.82it/s]

...

Testing: 100%|█████████▉| 1666/1669 [01:37<00:00, 17.58it/s]

SOURCE: [['1tzh_B_A_W'], ['vascular endothelial growth factor a'], ['EVQLVESGGGLVQPGGSLRLSCAASGFDIYDDDIHWVRQAPGKGLEWVAYIAPSYGYTDYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCSRSSDASYSYSAMDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSCDKTH|DIQMTQSPSSLSASVGDRVTITCRASQASYSSVAWYQQKPGKAPKLLIYAASYLYSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQSSASPATFGQGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGEC'], ['GQNHHEVVKFMDVYQRSYCHPIETLVDIFQEYPDEIEYIFKPSCVPLMRCGGCCNDEGLECVPTEESNITMQIMRIKPHQGQHIGEMSFLQHNKCECRPKKD']]
TARGET: ['ACACCCACCGCAGGCAGACGCAACGCCTCGGAGACTAGCC']
TARGET NAME: [['APIPred_1347']]
PREDICTED: ['TACTTCATAGAGTCGAGCAGCTGACTTGTAGAGACGATTTCACGTTCCGTTCCGTTCCGTTCCGTTCCGTTCCGTTCCGTTCCGTTCCGTTCCGTTCCGTTCCGTTCCGTTCCGTTCCGTTCCGTGAGCCGAGCCGAGCCGAGCCGAGCCGAGCCTCCGTTCCGTTCCGTTCCGTTCCGTTCCGTTCCGTTCCGT']
loss 7.282001495361328
TARGET:    ACACCCACCGCAGGCAGACGCAACGCCTCGGAGACTAGCC
PREDI

Testing: 100%|██████████| 1669/1669 [01:37<00:00, 17.06it/s]

SOURCE: [['6zxn_F_C'], ['spike glycoprotein'], ['MAQVQLVETGGGLVQPGGSLRLSCAASGFTFSSVYMNWVRQAPGKGPEWVSRISPNSGNIGYTDSVKGRFTISRDNAKNTLYLQMNNLKPEDTALYYCAIGLNLSSSSVRGQGTQVTVSSGGLPETGGHHHHHH|'], ['MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPGSASSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPS

7.3635962179422805

In [18]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):
    
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        #"train_acc": [],
        "test_loss": [],
        #"test_acc": [],
        "train_avg_levenshtein": []
    }
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_avg_levenshtein, train_normalized_levenshtein = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_avg_levenshtein, test_normalized_levenshtein = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)
        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            #f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            #f"test_acc: {test_acc:.4f}" 
            f"train_avg_levenshtein: {train_avg_levenshtein} | "
            f"train_normalized_levenshtein: {train_normalized_levenshtein} | "
            f"test_avg_levenshtein: {test_avg_levenshtein} | "  
            f"test_normalized_levenshtein: {test_normalized_levenshtein} | "    
        )

        # 5. Update results dictionary
        # Ensure all data is moved to CPU and converted to float for storage
        results["train_loss"].append(train_loss.item() if isinstance(train_loss, torch.Tensor) else train_loss)
        results["train_avg_levenshtein"].append(train_avg_levenshtein.item() if isinstance(train_avg_levenshtein, torch.Tensor) else train_avg_levenshtein)
        results["train_normalized_levenshtein"].append(train_normalized_levenshtein.item() if isinstance(train_normalized_levenshtein, torch.Tensor) else train_normalized_levenshtein)
        results["test_loss"].append(test_loss.item() if isinstance(test_loss, torch.Tensor) else test_loss)
        results["test_avg_levenshtein"].append(test_avg_levenshtein.item() if isinstance(test_avg_levenshtein, torch.Tensor) else test_avg_levenshtein)
        results["test_normalized_levenshtein"].append(test_normalized_levenshtein.item() if isinstance(test_normalized_levenshtein, torch.Tensor) else test_normalized_levenshtein)

    # 6. Return the filled results at the end of the epochs
    return results

/mnt/tank/scratch/azaikina/Model/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
# Set random seeds
torch.manual_seed(42) 
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 1

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Train model_0 
model_results = train(model=model, 
                        train_dataloader=train_dataloader_custom,
                        test_dataloader=test_dataloader_custom,
                        optimizer=optimizer,
                        loss_fn=loss_fn, 
                        epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_2392007/1113197593.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  decoder_input = torch.tensor(batch['decoder_input']).to(device)
/tmp/ipykernel_2392007/1113197593.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(batch['label']).to(device)

  0%|          | 0/1 [00:00<?, ?it/s]             

loss tensor(7.5328, grad_fn=<NllLossBackward0>)
LEVENSTEIN:180
NORM_LEVENSTEIN:5.14



  0%|          | 0/1 [00:00<?, ?it/s]                     

loss tensor(1.8828, grad_fn=<NllLossBackward0>)
LEVENSTEIN:33
NORM_LEVENSTEIN:0.38


...

loss tensor(0.2442, grad_fn=<NllLossBackward0>)
LEVENSTEIN:12
NORM_LEVENSTEIN:0.14



  0%|          | 0/1 [00:10<?, ?it/s]                      

loss tensor(1.2460, grad_fn=<NllLossBackward0>)
LEVENSTEIN:19
NORM_LEVENSTEIN:0.23


  0%|          | 0/1 [00:11<?, ?it/s]


KeyboardInterrupt: 

In [20]:
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)
  

save_model(model=model, target_dir='/mnt/tank/scratch/azaikina/Model/new_scripts_5_10', model_name='test.pth')

[INFO] Saving model to: /mnt/tank/scratch/azaikina/Model/new_scripts_5_10/test.pth


In [ ]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Transformer, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):
    
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)
        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # 5. Update results dictionary
        # Ensure all data is moved to CPU and converted to float for storage
        results["train_loss"].append(train_loss.item() if isinstance(train_loss, torch.Tensor) else train_loss)
        results["train_acc"].append(train_acc.item() if isinstance(train_acc, torch.Tensor) else train_acc)
        results["test_loss"].append(test_loss.item() if isinstance(test_loss, torch.Tensor) else test_loss)
        results["test_acc"].append(test_acc.item() if isinstance(test_acc, torch.Tensor) else test_acc)

    # 6. Return the filled results at the end of the epochs
    return results



Обучение
Добавить метрики (Левенштейнб нормализованный)
Сохранение модели
Логирование

Эмбеддинги
Другие attentions